In [ ]:
from pyspark.sql.window import Window
import pyspark.sql.functions as F

In [ ]:
#AWS cred locations
dbutils.fs.ls("/FileStore/tables")

# Read Credentials

In [ ]:
file_type = "csv"
first_row_is_header = "true"
delimiter = ","

# Read the CSV file to spark dataframe
aws_keys_df = spark.read.format(file_type)\
                    .option("header", first_row_is_header)\
                    .option("sep", delimiter)\
                    .load("/FileStore/tables/authentication_credentials.csv")

# Get the AWS access key and secret key from the spark dataframe
ACCESS_KEY = aws_keys_df.where(col('User name')=='databricks-user').select('Access key ID').collect()[0]['Access key ID']
SECRET_KEY = aws_keys_df.where(col('User name')=='databricks-user').select('Secret access key').collect()[0]['Secret access key']
# Encode the secrete key
ENCODED_SECRET_KEY = urllib.parse.quote(string=SECRET_KEY, safe="")


# Mount S3

In [ ]:
# AWS S3 bucket name
AWS_S3_BUCKET = "user-12853887c065-bucket"
# Mount name for the bucket
MOUNT_NAME = "/mnt/mount_name"
# Source url
SOURCE_URL = "s3n://{0}:{1}@{2}".format(ACCESS_KEY, ENCODED_SECRET_KEY, AWS_S3_BUCKET)
# Mount the drive
# dbutils are not supported outside of databricks notebook
dbutils.fs.mount(SOURCE_URL, MOUNT_NAME) # RUN ONCE ONLY!

display(dbutils.fs.ls("/mnt/mount_name/topics/12853887c065.geo/partition=0/"))

dbutils.fs.unmount("/mnt/mount_name") #unmount if it is already mounted

# Hadoop config and streaming from S3

In [ ]:
# Adding the packages required to get data from S3  
os.environ["PYSPARK_SUBMIT_ARGS"] = "--packages com.amazonaws:aws-java-sdk-s3:1.12.490,org.apache.hadoop:hadoop-aws:3.3.1 pyspark-shell"

# Configure the hadoop setting to read from the S3 bucket
hadoopConf.set('fs.s3a.access.key', ACCESS_KEY)
hadoopConf.set('fs.s3a.secret.key', SECRET_KEY)
hadoopConf.set('spark.hadoop.fs.s3a.aws.credentials.provider', 'org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider') # Allows the package to authenticate with AWS

# Stream from the S3 bucket
s3_bucket_url = "s3n://user-12853887c065-bucket/topics/12853887c065"
extension = "/partition=0/12853887c065"
while True:
    df_pin = spark.read.json(f"{s3_bucket_url}.pin{extension}.pin+0+*.json")
    df_geo = spark.read.json(f"{s3_bucket_url}.geo{extension}.geo+0+*.json")
    df_user = spark.read.json(f"{s3_bucket_url}.user{extension}.user+0+*.json")
    df_pin.show(5, True)
    df_geo.show(5, True)
    df_user.show(5, True)

# Cleaning Pin post

In [ ]:

#Check for null values
df_pin.filter(f.greatest(*[f.col(i).isNull() for i in df_pin.columns])).show()

# Perform the necessary transformations on the follower_count to ensure every entry is a number. Make sure the data type of this column is an int.
df_pin = df_pin.withColumn("follower_count", regexp_replace(df_pin["follower_count"], 'k', ''))

#Cast Data types on columns
df_pin = df_pin.withColumn("follower_count",col("follower_count").cast("int"))
df_pin = df_pin.withColumn("downloaded",col("downloaded").cast("int"))
df_pin = df_pin.withColumn("index",col("index").cast("int"))

#Update the follower_count to 1000
df_pin = df_pin.withColumn('follower_count', df_pin.follower_count*1000)

# Clean the data in the save_location column to include only the save location path
df_pin = df_pin.withColumn('save_location', split(df_pin.save_location, ' ').getItem(3))

# Rename the index column to ind.
df_pin = df_pin.withColumnRenamed("index","ind")

# Reorder the DataFrame columns to have the following column order:
new_column = ['ind', 'unique_id', 'title', 'description', 'follower_count', 'poster_name', 'tag_list', 'is_image_or_video', 'image_src', 'save_location', 'category']
df_pin = df_pin.select(new_column)

df_pin.show(5)

# Cleaning geolocation

In [ ]:
#Create a new column coordinates that contains an array based on the latitude and longitude columns
df_geo = df_geo.withColumn('coordinates', array('latitude', 'longitude'))

#Drop the latitude and longitude columns from the DataFrame
df_geo = df_geo.drop('latitude', 'longitude')

#Convert the timestamp column from a string to a timestamp data type
df_geo = df_geo.withColumn('timestamp', col('timestamp').cast('timestamp'))

#Reorder the DataFrame columns to have the following column order:
col_order = ['ind', 'country', 'coordinates', 'timestamp']
df_geo = df_geo.select(col_order)

df_geo.show(5)

# Cleaning Users Info

In [ ]:
#Create a new column user_name that concatenates the information found in the first_name and last_name columns
df_user = df_user.withColumn('user_name', concat_ws(' ', 'first_name', 'last_name'))

#Drop the first_name and last_name columns from the DataFrame
# df_user = df_user.drop('first_name', 'last_name')

#Convert the date_joined column from a string to a timestamp data type
df_user = df_user.withColumn('date_joined', col('date_joined').cast('timestamp'))

# Reorder the DataFrame columns to have the following column order:
user_col = ['ind', 'user_name', 'age', 'date_joined']
df_user = df_user.select(user_col)
df_user.show(5)

# Task 4

In [ ]:
# Find the most popular Pinterest category people post to based on their country.
# First join the dataframe
df_pin_geo = df_pin.join(df_geo, df_pin.ind==df_geo.ind, 'inner')

#Groupby country and category
df_pin_geo.groupby('country','category') \
    .agg(count('category')\
        .alias('category_count'))\
    .sort('category_count', ascending=False)\
    .show()

# df_pin_geo.show()

# Task 5 with SQL

In [ ]:
# Find how many posts each category had between 2018 and 2022.

#Create temporary view 
df_pin_geo.createOrReplaceTempView("pin_geo")

# Use spark sql queries
spark.sql('SELECT YEAR(timestamp) as post_year,\
                    category,\
                    count(category) category_count\
           FROM pin_geo\
            WHERE YEAR(timestamp) BETWEEN 2018 AND 2022\
            GROUP BY post_year, category\
            ORDER BY post_year, category_count DESC')\
            .show(50)

# Task 5 with window function

In [ ]:
# Find how many posts each category had between 2018 and 2022.

windowSpec = Window.partitionBy(df_pin_geo.category)\
                    .orderBy(df_pin_geo.timestamp)

df_pin_geo.filter((year('timestamp') >= 2018) & (year('timestamp') <= 2022))\
            .select(year('timestamp').alias('post_year'), 'category', count('category').over(windowSpec).alias('category_count')).show(100)

# Task 6 with SQL

In [ ]:
#Find the user with most followers

# Step 1: For each country find the user with the most followers.
# df_country_poster_follower = df_pin_geo.select('country', 'poster_name', 'follower_count')

spark.sql("WITH most_follower_table AS (\
                SELECT pin_geo.country, \
                    pin_geo.poster_name, \
                    SUM(pin_geo.follower_count) total_follower\
                FROM pin_geo\
                GROUP BY pin_geo.country, pin_geo.poster_name\
        )\
    SELECT country,\
        sum(total_follower) follower_count\
    FROM most_follower_table\
    GROUP BY country\
    ORDER BY follower_count DESC\
    LIMIT 1").show()

# Task 6 with Groupby

In [ ]:
df_pin_geo.select(df_pin_geo.country, df_pin_geo.poster_name, df_pin_geo.follower_count)\
            .groupby(df_pin_geo.country)\
            .agg(sum(df_pin_geo.follower_count).alias('follower_count'))\
            .orderBy(col('follower_count').desc()).show(1)

# Task 7 with SQL

In [ ]:
# What is the most popular category people post to based on the following age groups: 18-24, 25-35, 36-50, +50
#Join df_pin and df_user
df_pin_user = df_pin.join(df_user, df_pin.ind==df_user.ind, 'inner')

#Create temporary view 
df_pin_user.createOrReplaceTempView("pin_user")
spark.sql("SELECT\
    CASE\
        WHEN age BETWEEN 18 AND 24 THEN '18-24' \
        WHEN age BETWEEN 25 AND 35 THEN '25-35'\
        WHEN age BETWEEN 36 AND 50 THEN '36-50'\
        WHEN age > 50 THEN '50+'\
    END AS age_group,\
    category,\
    COUNT(category) AS category_count\
    FROM pin_user\
    GROUP BY age_group, category\
    ORDER BY age_group ASC, category_count DESC").show()

In [ ]:
spark.sql("\
        WITH category_table AS (\
            SELECT\
                CASE\
                    WHEN age BETWEEN 18 AND 24 THEN '18-24' \
                    WHEN age BETWEEN 25 AND 35 THEN '25-35'\
                    WHEN age BETWEEN 36 AND 50 THEN '36-50'\
                    WHEN age > 50 THEN '50+'\
                END AS age_group,\
                category,\
                COUNT(category) category_count\
            FROM pin_user\
            GROUP BY age_group, category\
            )\
        SELECT\
            age_group,\
            category,\
            MAX(category_count)\
        FROM category_table\
        GROUP BY age_group, category"
    ).show()

# Task 7 with Window function

In [ ]:
# What is the most popular category people post to based on the following age groups: 18-24, 25-35, 36-50, +50

#Add age-group col
df_pin_user = df_pin_user.withColumn('age_group', when((df_pin_user.age >= 18) & (df_pin_user.age <=25), '18-25')
                                                    .when((df_pin_user.age >= 26) & (df_pin_user.age <= 35), '26-35')
                                                    .when((df_pin_user.age >= 36) & (df_pin_user.age <= 50), '36-50')
                                                    .otherwise('50+'))

#Window specification
pin_user_windowSpec = Window.partitionBy('age_group')\
                            .orderBy('category')



df_pin_user.select('age_group', 'category', count(col('category')).over(pin_user_windowSpec).alias('category_count')).show()


# Task 8 with SQL

In [ ]:
# What is the median follower count for users in the following age groups: 18-24, 25-35, 36-50, +50

spark.sql("\
    SELECT\
            CASE\
                WHEN age BETWEEN 18 AND 24 THEN '18-24' \
                WHEN age BETWEEN 25 AND 35 THEN '25-35'\
                WHEN age BETWEEN 36 AND 50 THEN '36-50'\
                WHEN age > 50 THEN '50+'\
            END AS age_group,\
            round(mean(follower_count), 3) mean_follower_count,\
            max(follower_count) min_follower_count,\
            min(follower_count) min_follower_count\
        FROM pin_user\
        GROUP BY age_group\
        ORDER BY age_group\
    ").show()


# Task 8 with Groupby

In [ ]:
df_pin_user.groupby('age_group')\
            .agg(sum('follower_count').alias('total_follower_count'), round(mean('follower_count'), 3).alias('mean_follower_count')).show()

# Task 9

In [ ]:
#Find how many users have joined between 2015 and 2020.

# Join user and geo df
df_geo_user = df_geo.join(df_user, df_geo.ind==df_user.ind, 'inner')

# df_geo_user.withColumn('timestamp', YEAR(col('timestamp'))).show()

df_geo_user.groupby(year('timestamp').alias('post_year'))\
            .agg(count(year('timestamp')).alias('number_users_joined'))\
            .sort('number_users_joined')\
            .show()

# Task 10

In [ ]:
# Find the median follower count of users have joined between 2015 and 2020.
df_pin_geo.filter((year('timestamp') >= 2015) & (year('timestamp') <= 2020))\
            .groupby(year('timestamp').alias('post_year')).agg(round(mean('follower_count') ,2).alias('median_follower_count'))\
            .orderBy('post_year').show() #median function not defined. Testing with mean

# Task 11

In [ ]:
#Find the median follower count of users that have joined between 2015 and 2020, based on which age group they are part of.
# Join pin, geo & user df
df_pin_geo_user = df_pin.join(df_geo, df_pin.ind==df_geo.ind, 'inner')\
                        .join(df_user, df_geo.ind==df_user.ind, 'inner')

#Add age-group col
df_pin_geo_user = df_pin_geo_user.withColumn('age_group', when((df_pin_geo_user.age >= 18) & (df_pin_geo_user.age <=25), '18-25')
                                                            .when((df_pin_geo_user.age >= 26) & (df_pin_geo_user.age <= 35), '26-35')
                                                            .when((df_pin_geo_user.age >= 36) & (df_pin_geo_user.age <= 50), '36-50')
                                                            .otherwise('50+'))
df_pin_geo_user.filter((year('timestamp') >= 2015) & (year('timestamp') <= 2020))\
            .groupby('age_group',year('timestamp').alias('post_year')).agg(mean('follower_count').alias('median_follower_count')).show()

# Task 11 with Window function

In [ ]:
windowSpec = Window.partitionBy('age_group')\
                    .orderBy('timestamp')

# median function not imported, testing with mean
df_pin_geo_user.filter((year('timestamp') >= 2015) & (year('timestamp') <= 2020))\
                .select('age_group', year('timestamp').alias('post_year'), mean('follower_count').over(windowSpec).alias('mean_follower_count')).show()